<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/DeployToProduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📥 Step 1: Upload the `.env` File

This notebook requires a `.env` file containing your API keys and credentials. Create a file with the following environment variables:

| Variable Name | Purpose |
|---|---|
| `OPENAI_API_KEY` | Access to OpenAI API |
| `AWS_ACCESS_KEY_ID` | AWS access key |
| `AWS_SECRET_ACCESS_KEY` | AWS secret access key |
| `AWS_DEFAULT_REGION` | AWS REGION |
| `S3_BUCKET_NAME` | Your S3 bucket name |
| `IMAGE_FOLDER_NAME` | Image directory |
| `IMAGES_DIR` | Directory path for images |
| `ELEPHANT_PRIVATE_KEY` | Elephant wallet private key |
| `PINATA_JWT` | Pinata authentication token |
|`NETLIFY_SITE_ID` | Netlify project ID |
|`NETLIFY_TOKEN` | Netlify token |

### To upload:
1. Click the **folder icon** 📂 in the left sidebar
2. Click the **"Upload"** button
3. Select your `.env` file

### Example `.env` file:
```env
OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
AWS_ACCESS_KEY_ID=XXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXX
S3_BUCKET_NAME=your-s3-bucket-name-here
IMAGES_DIR=images
IMAGE_FOLDER_NAME=images
ELEPHANT_PRIVATE_KEY=xxxxx
PINATA_JWT=xxxxx
NETLIFY_SITE_ID=xxxxx
NETLIFY_TOKEN=xxxxxxxx

```

> ⚠️ **Security Note:** Never commit your `.env` file to version control or share it publicly.


## Step 2: Upload upload-results.csv

Upload the upload-results.csv file from any step that you want to push the HTML to production.

👉 Make sure to upload it to the content/ folder. This is required for the production pipeline to pick it up properly.


In [ ]:
!ls -la /content/upload-results.csv

## Step 3 Set up to be able to deploy to production


In [ ]:
%%bash

# Elephant Fact Sheet Template Installer (Silent)
set -e  # Exit on any error

INSTALL_DIR="${HOME}/.elephant-fact-sheet"
BIN_DIR="${HOME}/.local/bin"
STATIC_TARGET_DIR="$BIN_DIR/templates/assets/static"
TEST="/content/templates/assets/static"

# Check Node.js version
if ! command -v node &> /dev/null; then
    exit 1
fi

NODE_VERSION=$(node -v | cut -d'v' -f2 | cut -d'.' -f1)
if [ "$NODE_VERSION" -lt 18 ]; then
    exit 1
fi

# Check npm
if ! command -v npm &> /dev/null; then
    exit 1
fi

# Clean up any existing installation
rm -rf "$INSTALL_DIR"

# Clone repo silently
git clone --quiet https://github.com/elephant-xyz/fact-sheet-template.git "$INSTALL_DIR"

cd "$INSTALL_DIR"

# Silent npm install and build
npm install --silent --no-audit --no-fund > /dev/null 2>&1
npm run build --silent > /dev/null 2>&1

# Setup directories and copy assets
mkdir -p "$BIN_DIR" "$STATIC_TARGET_DIR" "$TEST"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$STATIC_TARGET_DIR"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$TEST"

# Link and make executable
ln -sf "$INSTALL_DIR/bin/fact-sheet.js" "$BIN_DIR/fact-sheet"
chmod +x "$INSTALL_DIR/bin/fact-sheet.js"

## Step 2

In [38]:
!rm -rf ~/.elephant-fact-sheet
!curl -fsSL https://raw.githubusercontent.com/elephant-xyz/fact-sheet-template/main/install.sh | bash
!cp .env ~/.elephant-fact-sheet/
!node ~/.elephant-fact-sheet/bin/deploy-from-csv.js "/content/upload-results.csv"

📋 Found 2 entries to deploy from upload-results.csv

📦 Processing: Property bafkreifhew5pcqelu3t7viyasm5fbzuel557jluylabexh2m3s6yzw6fha (dataCid: bafkreifhew5pcqelu3t7viyasm5fbzuel557jluylabexh2m3s6yzw6fha) -> http://dweb.link/ipfs/bafybeiazra3mxg6ipcjzmxxkr4eogwf2yexdf57s6mtffqw75xmuqf6pdq
🔍 Debug: Script path: /root/.elephant-fact-sheet/bin/deploy-from-csv.js
🔍 Debug: Script dir: /root/.elephant-fact-sheet/bin
🔍 Debug: Fact sheet dir: /root/.elephant-fact-sheet
🔍 Debug: Deploy script path: /root/.elephant-fact-sheet/bin/deploy-production.js
🚀 Deploying property bafkreifhew5pcqelu3t7viyasm5fbzuel557jluylabexh2m3s6yzw6fha from http://dweb.link/ipfs/bafybeiazra3mxg6ipcjzmxxkr4eogwf2yexdf57s6mtffqw75xmuqf6pdq...
✅ Successfully deployed property bafkreifhew5pcqelu3t7viyasm5fbzuel557jluylabexh2m3s6yzw6fha
📦 Netlify CLI not found, installing...
✅ Netlify CLI installed successfully
🚀 Deploying property bafkreifhew5pcqelu3t7viyasm5fbzuel557jluylabexh2m3s6yzw6fha to elephant.xyz production
📥 D

In [24]:
#!ls -al


total 88
drwxr-xr-x 1 root root  4096 Jul 30 19:35 .
drwxr-xr-x 1 root root  4096 Jul 30 18:47 ..
drwxr-xr-x 4 root root  4096 Jul 29 13:36 .config
-rw-r--r-- 1 root root  1387 Jul 30 19:35 deploy-production-error.log
-rw-r--r-- 1 root root 51753 Jul 30 19:35 deploy-production.log
drwxr-xr-x 3 root root  4096 Jul 30 19:35 deploy-temp
-rw-r--r-- 1 root root  1283 Jul 30 19:18 .env
drwxr-xr-x 2 root root  4096 Jul 30 19:17 .ipynb_checkpoints
drwxr-xr-x 1 root root  4096 Jul 29 13:36 sample_data
-rw-r--r-- 1 root root   933 Jul 30 19:17 upload-results.csv
